In [ ]:
import xarray as xr
import pathlib as pl
import pandas as pd
import pywatershed
import os
import dask
import numpy as np

In [ ]:
all_models = ['01473000','05431486','09112500','14015000']# Create a list of all cutouts

In [ ]:
rootdir = pl.Path('../NHM_extractions/20230110_pois_haj/')# Path to location of cutouts

In [ ]:
var_output_files = ['hru_actet.nc', 'recharge.nc', 'soil_rechr.nc', 'snowcov_area.nc', 'seg_outflow.nc',]#output files of interest

### Working currently from a single cutout directory

In [ ]:
cm = all_models[0] # sets cutout from list
outvardir = rootdir/ cm / 'output'# stes path to location of NHM output folder where output files are.

In [ ]:
# set the file name for the postprocessed model output file that PEST will read
of_name = 'alloutput.dat'# name of file

In [ ]:
########### Useful code
#indat
#df.index.dtypes
#xr.__version__

### Actual ET
#### Get and check the daily data

In [ ]:
actet_daily = xr.open_mfdataset(outvardir / 'hru_actet.nc')

#### Post-process daily output to match observation targets of "monthly" and "mean monthly"

In [ ]:
#Creates a dataframe time series of monthly values (average daily rate for the month)
actet_monthly = actet_daily.resample(time = 'm').mean()

In [ ]:
# Creates a dataframe time series of mean monthly (mean of all jan, feb, mar....)
actet_mean_monthly = actet_monthly.groupby('time.month').mean()

In [ ]:
### Used for plotting/checking
#df = actet_monthly.to_dataframe()# makes a dataframe for plotting and writing out
#df_r = np.ravel(actet_monthly_df, order = 'C')# flattens the 2D array to a 1D array--just playing
#df[8465:8473]
#df.xs(5625, level=1).plot()

#### Now write values to the template file

In [ ]:
inds = [f'{i.year}_{i.month}:{j}' for i in actet_monthly.indexes['time'] for j in actet_monthly.indexes['nhm_id']]# set up the indices in sequence
varvals = np.ravel(actet_monthly.to_array(), order = 'C')# flattens the 2D array to a 1D array--just playing 

with open(rootdir / cm / of_name, encoding="utf-8", mode='w') as ofp:
    [ofp.write(f'actet_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [ ]:
inds = [f'{i}:{j}' for i in actet_mean_monthly.indexes['month'] for j in actet_mean_monthly.indexes['nhm_id']]
varvals =  np.ravel(actet_mean_monthly.to_array(), order = 'C')# flattens the 2D array to a 1D array 

with open(rootdir / cm / of_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'actet_mean_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

### Post Process recharge for calibration use
#### Get daily output file from NHM for recharge

In [ ]:
recharge_daily = xr.open_mfdataset(outvardir / 'recharge.nc')

#### Post-process daily output to match observation target of "annual recharge" as an average daily rate for the year

In [ ]:
recharge_annual = recharge_daily.resample(time = 'Y').mean()

#### Write values to template file

In [ ]:
inds = [f'{i.year}:{j}' for i in recharge_annual.indexes['time'] for j in recharge_annual.indexes['nhm_id']]
varvals =  np.ravel(recharge_annual.to_array(), order = 'C')# flattens the 2D array to a 1D array 

with open(rootdir / cm / of_name, encoding="utf-8",mode='a') as ofp:
    [ofp.write(f'recharge_ann:{i}          {j}\n') for i,j in zip(inds,varvals)]

### Post Process "soil_rechr" to compare to target
#### Get daily output file from NHM for soil recharge

In [ ]:
soil_rechr_daily = xr.open_mfdataset(outvardir / 'soil_rechr.nc')

In [ ]:
#Creates a dataframe time series of monthly values (average daily rate for each month)
soil_rechr_monthly = soil_rechr_daily.resample(time = 'm').mean() 

#Creates a dataframe time series of annual values (average daily value for each year)
soil_rechr_annual = soil_rechr_daily.resample(time = 'Y').mean() 

In [ ]:
inds = [f'{i.year}_{i.month}:{j}' for i in soil_rechr_monthly.indexes['time'] for j in soil_rechr_monthly.indexes['nhm_id']]
varvals = np.ravel(soil_rechr_monthly.to_array(), order = 'C')# flattens the 2D array to a 1D array

with open(rootdir / cm / of_name, encoding="utf-8",mode='a') as ofp:
    [ofp.write(f'soil_rechr_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [ ]:
inds = [f'{i.year}:{j}' for i in soil_rechr_annual.indexes['time'] for j in soil_rechr_annual.indexes['nhm_id']]
varvals =  np.ravel(soil_rechr_annual.to_array(), order = 'C')# flattens the 2D array to a 1D array 

with open(rootdir / cm / of_name, encoding="utf-8",mode='a') as ofp:
    [ofp.write(f'soil_rechr_ann:{i}          {j}\n') for i,j in zip(inds,varvals)]

### Post Process "snowcov_area" to compare to target
#### Get and check the daily data

In [ ]:
snowcov_area_daily = xr.open_mfdataset(outvardir / 'snowcov_area.nc')

In [ ]:
inds = [f'{i.year}_{i.month}_{i.day}:{j}' for i in snowcov_area_daily.indexes['time'] for j in snowcov_area_daily.indexes['nhm_id']]
varvals = np.ravel(snowcov_area_daily.to_array(), order = 'C')# flattens the 2D array to a 1D array

with open(rootdir / cm / of_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'sca_daily:{i}          {j}\n') for i,j in zip(inds,varvals)]

### Post Process "seg_outflow" to compare to target
#### Get and check the daily data

### Create model output template file

In [ ]:
oftemplate_name = 'alloutput_template.dat'

#### AET

In [ ]:
inds = [f'{i.year}_{i.month}:{j}' for i in actet_monthly.indexes['time'] for j in actet_monthly.indexes['nhm_id']]# set up the indices in sequence
varvals = np.ravel(actet_monthly.to_array(), order = 'C')# flattens the 2D array to a 1D array--just playing 

with open(rootdir / cm / oftemplate_name, encoding="utf-8", mode='w') as ofp:
    [ofp.write(f'actet_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [ ]:
inds = [f'{i}:{j}' for i in actet_mean_monthly.indexes['month'] for j in actet_mean_monthly.indexes['nhm_id']]
varvals =  np.ravel(actet_mean_monthly.to_array(), order = 'C')# flattens the 2D array to a 1D array 

with open(rootdir / cm / oftemplate_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'actet_mean_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]